In [1]:
%pip install torch torchvision torchaudio
%pip install torch-geometric

In [2]:
import pandas as pd
import joblib
import numpy as np
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv
import torch.nn.functional as F

original_data = pd.read_csv("C:/Users/jyc37/UWI/CurrentCourses/BIgData/Project/gnndata/processed_full.csv")
labels = original_data['isFraud']


original_data.head(5)


,0,1,2,3,4,5,6,7,8,9,...,349,350,351,352,353,354,355,356,357,isFraud
0,-1.577987,-0.278167,0.821695,-0.009783,-0.281425,-1.396380,0.243321,0.069833,-0.138579,-0.098020,...,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0,0
1,-1.577986,-0.443327,-1.457558,0.264810,-0.281425,-2.368254,0.347543,0.069833,-0.183967,-0.098020,...,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0,0
2,-1.577972,-0.317889,-1.068263,0.813997,-0.281425,-0.813255,0.399654,0.069833,0.967246,-0.098020,...,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0,0
3,-1.577965,-0.355521,1.679858,1.305711,-0.281425,-2.003802,1.921291,0.069833,-0.183967,-0.090533,...,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0,0
4,-1.577964,-0.355521,-1.102133,0.967258,-0.281425,-2.368254,1.337649,0.069833,-0.183967,-0.098020,...,1.0,1.0,124.0,1.0,0.0,1.0,1.0,2.0,954.0,0


In [3]:
features = original_data.drop( columns= ['isFraud'])
feature_columns = joblib.load('C:/Users/jyc37/UWI/CurrentCourses/BIgData/Project/gnndata/feature_columns.pkl')
features.columns = feature_columns

features.head(5)

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,-1.577987,-0.278167,0.821695,-0.009783,-0.281425,-1.396380,0.243321,0.069833,-0.138579,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
1,-1.577986,-0.443327,-1.457558,0.264810,-0.281425,-2.368254,0.347543,0.069833,-0.183967,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
2,-1.577972,-0.317889,-1.068263,0.813997,-0.281425,-0.813255,0.399654,0.069833,0.967246,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
3,-1.577965,-0.355521,1.679858,1.305711,-0.281425,-2.003802,1.921291,0.069833,-0.183967,-0.090533,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
4,-1.577964,-0.355521,-1.102133,0.967258,-0.281425,-2.368254,1.337649,0.069833,-0.183967,-0.098020,...,1.0,1.0,1.0,124.0,1.0,0.0,1.0,1.0,2.0,954.0


In [4]:
labels.value_counts()

isFraud
0    569877
1     20663
Name: count, dtype: int64

In [5]:
fraud_data = features[labels == 1].copy()
fraud_data.shape

(20663, 358)

In [6]:
fraud_data.head(5)

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
203,-1.577259,1.296077,1.707607,1.407885,-0.281425,0.644557,-0.423698,0.069833,-0.183967,-0.090533,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
240,-1.577165,-0.409467,0.717026,-1.657343,2.809740,-1.517864,0.076566,0.069833,-0.183967,-0.105507,...,0.0,0.0,0.0,29.0,0.0,0.0,1.0,1.0,2.0,916.0
243,-1.577154,-0.409467,0.717026,-1.657343,2.809740,-1.517864,0.076566,0.069833,-0.183967,-0.098020,...,0.0,0.0,0.0,29.0,0.0,0.0,1.0,1.0,2.0,916.0
245,-1.577143,-0.409467,0.717026,-1.657343,2.809740,-1.517864,0.076566,0.069833,-0.183967,-0.090533,...,0.0,0.0,0.0,29.0,0.0,0.0,1.0,1.0,2.0,916.0
288,-1.576993,0.085690,1.362791,1.165221,2.809740,0.644557,0.076566,0.069833,-0.183967,-0.098020,...,2.0,1.0,1.0,46.0,0.0,0.0,0.0,0.0,2.0,1735.0


In [7]:
print(fraud_data.shape[1])

358


In [8]:
#%pip install tensorflow

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential

def build_generator(latent_dim, output_dim):
       model = Sequential([
        Dense(128, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(output_dim, activation='tanh')  # Ensure output_dim = 358
    ])
       return model


def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Hyperparameters
latent_dim = 100
input_dim = fraud_data.shape[1]
generator = build_generator(latent_dim, input_dim)
discriminator = build_discriminator(input_dim)

c:\Users\jyc37\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jyc37\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [10]:
gan = Sequential([generator, discriminator])
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002))
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002))

fraud_data_normalized = (fraud_data - fraud_data.mean()) / (fraud_data.std() + 1e-7)

epochs = 1000
batch_size = 128
for epoch in range(epochs):
    idx = np.random.randint(0, fraud_data_normalized.shape[0], batch_size)
    real_samples = fraud_data_normalized.iloc[idx].values
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_samples = generator.predict(noise, verbose=0)

    d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    if epoch %100 == 0:
        print(f"Epoch {epoch}, Discriminator Loss: {d_loss:.4f}, Generator Loss: {g_loss:.4f}")

c:\Users\jyc37\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, Discriminator Loss: 0.6899, Generator Loss: 0.7648
Epoch 100, Discriminator Loss: 2.3621, Generator Loss: 0.0857
Epoch 200, Discriminator Loss: 2.9261, Generator Loss: 0.0443
Epoch 300, Discriminator Loss: 3.2460, Generator Loss: 0.0299
Epoch 400, Discriminator Loss: 3.4722, Generator Loss: 0.0226
Epoch 500, Discriminator Loss: 3.6479, Generator Loss: 0.0182
Epoch 600, Discriminator Loss: 3.7923, Generator Loss: 0.0152
Epoch 700, Discriminator Loss: 3.9133, Generator Loss: 0.0131
Epoch 800, Discriminator Loss: 4.0181, Generator Loss: 0.0115
Epoch 900, Discriminator Loss: 4.1093, Generator Loss: 0.0102


In [11]:
num_synthetic_samples = len(labels) - (sum(labels) *2)
noise = np.random.normal(0, 1, (num_synthetic_samples, latent_dim))
synthetic_fraud = generator.predict(noise)

synthetic_fraud.reshape(num_synthetic_samples, fraud_data.shape[1])
synthetic_labels = pd.Series(1, index=range(num_synthetic_samples))

17163/17163 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step


In [12]:
synthetic_fraud_df = pd.DataFrame(synthetic_fraud, columns=feature_columns)
synthetic_fraud_df.head(5)
synthetic_fraud_df.shape

(549214, 358)

In [13]:
fraud_std = fraud_data.std() + 1e-7
synthetic_fraud_denorm = (synthetic_fraud_df * fraud_std.values) + fraud_data.mean()

In [14]:
balanced_features = pd.concat([features, pd.DataFrame(synthetic_fraud, columns = features.columns)])
balanced_labels = pd.concat([labels, synthetic_labels]).reset_index(drop=True)

In [15]:
balanced_features.shape
balanced_labels.value_counts()

0    569877
1    569877
Name: count, dtype: int64

In [16]:
balanced_features.head(10)

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,-1.577987,-0.278167,0.821695,-0.009783,-0.281425,-1.396380,0.243321,0.069833,-0.138579,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
1,-1.577986,-0.443327,-1.457558,0.264810,-0.281425,-2.368254,0.347543,0.069833,-0.183967,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
2,-1.577972,-0.317889,-1.068263,0.813997,-0.281425,-0.813255,0.399654,0.069833,0.967246,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
3,-1.577965,-0.355521,1.679858,1.305711,-0.281425,-2.003802,1.921291,0.069833,-0.183967,-0.090533,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
4,-1.577964,-0.355521,-1.102133,0.967258,-0.281425,-2.368254,1.337649,0.069833,-0.183967,-0.098020,...,1.0,1.0,1.0,124.0,1.0,0.0,1.0,1.0,2.0,954.0
5,-1.577963,-0.359702,-0.808325,1.229080,-0.281425,0.644557,-0.204832,0.069833,-0.068433,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
6,-1.577960,0.100237,0.491570,-0.016169,-0.281425,-0.813255,-1.726470,0.069833,-0.216977,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
7,-1.577959,1.201999,0.570531,0.813997,-0.281425,0.644557,0.347543,0.069833,-0.183967,-0.098020,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0
8,-1.577957,-0.501865,-1.447765,-1.676500,-0.281425,0.644557,0.472609,0.069833,-0.183967,-0.098020,...,1.0,1.0,1.0,99.0,1.0,0.0,0.0,1.0,2.0,1727.0
9,-1.577957,-0.075376,1.530302,-1.606256,-0.281425,0.595963,-0.913540,0.069833,-0.138579,-0.090533,...,2.0,2.0,2.0,96.0,2.0,2.0,2.0,2.0,1.0,1735.0


In [17]:
balanced_features.to_csv("balanced_features.csv", index = False)
balanced_labels.to_csv("balanced_labels.csv", index=False)